# INTERROGATION D'UNE BASE DOCUMENTAIRE 
## Oscar PERIANAYAGASSAMY - M2 IASD App. - 2025-2026

In [272]:
%pip install requests pymongo pandas jinja2 tqdm

Note: you may need to restart the kernel to use updated packages.


In [273]:
import requests
import json
from pathlib import Path
import logging
import pandas as pd
from tqdm import tqdm

logging.basicConfig(
    level=logging.INFO,
    format="[%(levelname)s] - %(asctime)s - %(name)s - %(message)s"
)

logger = logging.getLogger("__main__")

# 1. Choix et préparation des données

## 1.1) Recherche du fichier JSON

Source des données : ```data.police.uk```

> All the data on this site is made available under the Open Government Licence v3.0.

Autre source : ```openstreetmap.org```

> Les données OpenStreetMap sont sous licence Licence de base de données ouverte Open Data Commons (ODbL).

### Introduction

Le choix de la source pour les données s'est porté sur ```data.police.uk``` qui répertorie des données autour du crime et des actions policières en Angleterre, au Pays de Galle et en Ireland du Nord. Cette source propose un point d'API permettant donc une extraction au format JSON immédiate sur les éléments suivants : 

- *Neighbourhood team members*, i.e. des détails sur la composition des brigades de police de proximité. 
- *Upcoming events*, i.e. le dénouement de certaines affaires suivies ou de recherche d'individus.
- *Street-level crime and outcome data*, i.e. de la donnée sur les crimes localisée géographiquement (et approximativement) ainsi que sur la suite des événements d'un point de vue judiciaire.
- *Nearest police stations*, i.e. la localisation des postes de police à proximité d'une localisation précisée.

Les données sont disponibles sous la licence Open Government Licence v3.0 ce qui signifie que nous pouvons les utiliser librement, les croiser et les adapter selon les besoins notre application.

Pour l'étude, nous nous concentrons sur la partie *Street-level crime and outcome data*. Elle permet d'obtenir les résultats suivants :

<table border="1" class="dataframe">  
    <thead>    
        <tr style="text-align: right;">      
            <th>Tag</th>      
            <th>Description</th>    
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>category</td>
            <td>Categorie du crime</td>
        </tr>
        <tr>
            <td>persistent_id</td>
            <td>Identifiant unique de 64 caractères pour le crime (différent de l'autre attribut id qui peut varier dans le temps)</td>
        </tr>
        <tr>
            <td>month</td>
            <td>Mois du crime</td>
        </tr>
        <tr>
            <td>location</td>
            <td>Location approximative de l'incident</td>
        </tr>
        <tr>
            <td>location/latitude</td>
            <td>Lattitude</td>
        </tr>
        <tr>
            <td>location/street</td>
            <td>La rue approximée où le crime a eu lieu</td>
        </tr>
        <tr>
            <td>location/street/id</td>
            <td>Identifiant unique de la rue</td>
        </tr>
        <tr>
            <td>location/street/name</td>
            <td>Nom de l'endroit</td>
        </tr>
        <tr>
            <td>location/longitude</td>
            <td>Longitude</td>
        </tr>
        <tr>
            <td>context</td>
            <td>Informations additionnelles sur le crime</td>
        </tr>
        <tr>
            <td>id</td>
            <td>identifiant du crime (pour l'API seulement, ce n'est pas un vrai identifiant de police)</td>
        </tr>
        <tr>
            <td>location_type</td>
            <td>Typologie du lieu du crime : soit 'Force' ou 'BTP'. 'Force' fait référence à un lieu normal pour les forces de police; 'BTP' indique qu'il s'agit d'un lieu de la British Transport Police. A savoir que les lieux BTP sont aussi dans les lieux d'actop, de la police normale</td>
        </tr>
        <tr>
            <td>location_subtype</td>
            <td>Pour les lieux BTP, le type de lieu où le crime a été enregistré</td>
        </tr>
        <tr>
            <td>outcome_status</td>
            <td>Category et date de la dernière procédure concernant le crime</td>
        </tr>
        <tr>
            <td>outcome_status/category</td>
            <td>Categorie de la procédure (ex. 'fined' (verbalisé), 'imprisoned' (emprisonné) etc)</td>
        </tr>
        <tr>
            <td>outcome_status/date</td>
            <td>Date du changement</td>
        </tr>
        <tr>
            <td>outcomes</td>
            <td>Liste contenant les procédures associées</td>
        </tr>
        <tr>
            <td>outcomes/category</td>
            <td>Categorie de la procédure</td>
        </tr>
        <tr>
            <td>outcomes/category/code</td>
            <td>Code de la procédure</td>
        </tr>
        <tr>
            <td>outcomes/category/name</td>
            <td>Nom de la procédure</td>
        </tr>
        <tr>
            <td>outcomes/date</td>
            <td>Date de la procédure</td>
        </tr>
        <tr>
            <td>outcomes/person_id</td>
            <td>ID de la personne coupable si disponible</td>
        </tr>
    </tbody>
</table>

### Récupération des actes criminels perpétrés à Londres en novembre 2025 (dernier mois disponible en base)

L'API nous propose deux formats pour requêter sur ces données : 

- *Point-wise*, i.e. tous les crimes dans un rayon d'1 Mile (~1,6Km) autour de la localisation saisie sous le format $(\texttt{lattitude}, \texttt{longitude})$.
- *Area-wise*, i.e. tous les crimes dans un polygone spécifié sous le format $\{(\texttt{lattitude}_1, \texttt{longitude}_1),(\texttt{lattitude}_2, \texttt{longitude}_2),(\texttt{lattitude}_3, \texttt{longitude}_3),(\texttt{lattitude}_4, \texttt{longitude}_4)\}$ (avec au moins trois paires de coordonnées...)

Nous utilisons la deuxième méthode. Néanmoins, il a fallu recentrer assez précisément sur Londes car si l'API renvoie plus de 10 000 éléments, le code d'erreur 503 est levé et aucun résultat n'est transmis. Pour ce faire, nous choissisons arbitrairement de se placer au centre de la capitale outre-Manche.

Pour le calcul des coordonnées, nous utilisons ```openstreetmap.org```.


<figure>
    <img src="./images/carte.png"
         alt="image-carte-londres-osm">
    <figcaption><i>Carte recentrée sur Londres, depuis openstreetmap.</i></figcaption>
</figure>


Les coordonnées géographiques que le site web nous donne sont les suivantes : 


<figure>
    <img src="images/coordonnees1.png"
         alt="coord-londres"
         width=500
         height=400
         >
    <figcaption><i>Coordonnées géographiques de la carte de Londres affichée plus haut.</i></figcaption>
</figure>


Ainsi nous pouvons désormais récupérer le document JSON souhaité à l'adresse suivante par déduction des coordonnées fournies : 

`https://data.police.uk/api/crimes-street/all-crime?date=2[date]&poly=[lat1],[lng1]:[lat2],[lng2]:[lat3],[lng3]:[lat4],[lng4]`

avec 
- `[date] <- 2025-11` pour novembre 2025,
- `[lat1],[lng1]:[lat2],[lng2]:[lat3],[lng3]:[lat4],[lng4] <- 51.52669,-0.12634:51.52669,-0.06008:51.49763,-0.06008:51.49763,-0.12634` pour la zone rectangulaire que l'on souhaite couvrir.

In [274]:
""" # Définition du lien vers le point d'API
url = "https://data.police.uk/api/crimes-street/all-crime?date=2025-11&poly=51.52669,-0.12634:51.52669,-0.06008:51.49763,-0.06008:51.49763,-0.12634"

try:
    # méthode GET (sans header dans ce cas)
    response = requests.get(url)
except Exception as err:
    logger.error(err)

# Si code de retour est 200, cela signifie que les données ont bien été transmises
if response.status_code == 200:
    logger.info("L'appel API s'est déroulé correctement. Lecture du document JSON.")
    
    # Transformation de l'objet requests en objet JSON
    crime_data = response.json()

    logger.info(f"{len(crime_data)} enregistrements renvoyés.")

elif response.status_code == 404:
    logger.warning(f"Erreur {response.status_code}: la page n'a pas pu être atteint (vérifier l'url : {url})")
elif response.status_code == 503:
    logger.warning(f"Erreur {response.status_code} : le serveur a reçu la requête mais a généré plus de 10 000 résultats. Affiner la zone.") """

' # Définition du lien vers le point d\'API\nurl = "https://data.police.uk/api/crimes-street/all-crime?date=2025-11&poly=51.52669,-0.12634:51.52669,-0.06008:51.49763,-0.06008:51.49763,-0.12634"\n\ntry:\n    # méthode GET (sans header dans ce cas)\n    response = requests.get(url)\nexcept Exception as err:\n    logger.error(err)\n\n# Si code de retour est 200, cela signifie que les données ont bien été transmises\nif response.status_code == 200:\n    logger.info("L\'appel API s\'est déroulé correctement. Lecture du document JSON.")\n\n    # Transformation de l\'objet requests en objet JSON\n    crime_data = response.json()\n\n    logger.info(f"{len(crime_data)} enregistrements renvoyés.")\n\nelif response.status_code == 404:\n    logger.warning(f"Erreur {response.status_code}: la page n\'a pas pu être atteint (vérifier l\'url : {url})")\nelif response.status_code == 503:\n    logger.warning(f"Erreur {response.status_code} : le serveur a reçu la requête mais a généré plus de 10 000 résul

Pour venir enrichir ces données, nous allons utiliser la partie *Outcomes for a specific crime* qui nous donne la liste des suites pour chaque crime identifié par son `persistent_id`.

In [275]:
""" url = "https://data.police.uk/api/outcomes-for-crime/{}"

i = 1
for document in tqdm(crime_data):
    try:
        
        #  si le document a un persistent id
        if document["persistent_id"]:
            response = requests.get(url.format(document["persistent_id"]))
            
        else :
            continue

    except Exception as err:
        print("ee")
        logger.error(err)

    # Si code de retour est 200, cela signifie que les données ont bien été transmises
    if response.status_code == 200:
        #logger.info("L'appel API s'est déroulé correctement. Lecture du document JSON.")
        
        # Transformation de l'objet requests en objet JSON
        document["outcomes"] = response.json()["outcomes"]

    elif response.status_code == 404:
        logger.warning(f"Erreur {response.status_code}: la page n'a pas pu être atteint (vérifier l'url : {url})")
    elif response.status_code == 503:
        logger.warning(f"Erreur {response.status_code} : le serveur a reçu la requête mais a généré plus de 10 000 résultats. Affiner la zone.")
    elif response.status_code == 429:
        logger.warning(f"Erreur {response.status_code} : le serveur a reçu trop de requêtes dans un court lapse de temps. Attendre 10 secondes")
        time.sleep(10)
"""

' url = "https://data.police.uk/api/outcomes-for-crime/{}"\n\ni = 1\nfor document in tqdm(crime_data):\n    try:\n\n        #  si le document a un persistent id\n        if document["persistent_id"]:\n            response = requests.get(url.format(document["persistent_id"]))\n\n        else :\n            continue\n\n    except Exception as err:\n        print("ee")\n        logger.error(err)\n\n    # Si code de retour est 200, cela signifie que les données ont bien été transmises\n    if response.status_code == 200:\n        #logger.info("L\'appel API s\'est déroulé correctement. Lecture du document JSON.")\n\n        # Transformation de l\'objet requests en objet JSON\n        document["outcomes"] = response.json()["outcomes"]\n\n    elif response.status_code == 404:\n        logger.warning(f"Erreur {response.status_code}: la page n\'a pas pu être atteint (vérifier l\'url : {url})")\n    elif response.status_code == 503:\n        logger.warning(f"Erreur {response.status_code} : le se

(25min d'exécution pour le code ci-dessus, privilégier l'importation directe sans télécharger les données de nouveau)

Transformation du champ des coordonnées en nombre flottant pour faciliter les requêtes de calcul.

In [276]:
""" for document in crime_data:
    document["location"]["latitude"] = float(document["location"]["latitude"])
    document["location"]["longitude"] = float(document["location"]["longitude"]) 
"""

' for document in crime_data:\n    document["location"]["latitude"] = float(document["location"]["latitude"])\n    document["location"]["longitude"] = float(document["location"]["longitude"]) \n'

### Enregistrement des documents JSON

Nous enregistrons les données téléchargées dans le dossier `data` du projet.

In [277]:
""" # Définition du dossier où les données seront stockées
data_dir = Path() / "data"

# Création du dossier s'il n'existe pas
data_dir.mkdir(parents=True, exist_ok=True)

with open(data_dir / "base-crimes.json", "w") as json_file:

    # Tentative d'enregistrement du fichier
    try:
        json.dump(crime_data, json_file)
        logger.info(f"Le fichier base est crée à l'adresse {data_dir / "base-crimes.json"}.")

    except Exception as err:
        logger.error(f"{err}") """

' # Définition du dossier où les données seront stockées\ndata_dir = Path() / "data"\n\n# Création du dossier s\'il n\'existe pas\ndata_dir.mkdir(parents=True, exist_ok=True)\n\nwith open(data_dir / "base-crimes.json", "w") as json_file:\n\n    # Tentative d\'enregistrement du fichier\n    try:\n        json.dump(crime_data, json_file)\n        logger.info(f"Le fichier base est crée à l\'adresse {data_dir / "base-crimes.json"}.")\n\n    except Exception as err:\n        logger.error(f"{err}") '

## 1.2) Importation de la base dans MongoDB 

> Inspiré de l'article  "Connecting MongoDB to Jupyter Notebook" de `geeksforgeeks.org` pour toute la partie qui concerne la connexion à la base MongoDB depuis Python.

> Documentation PyMongo : `https://pymongo.readthedocs.io`

### Importation via le MongoShell

Suite d'opérations réalisées sous MacOS (il est probable que le comportement diffère légèrement pour les utilisateurs Windows).

- Exécuter la commande `mongosh` depuis le terminal à la racine du projet pour initialiser la connexion à la base (à l'adresse `localhost:27017`).
- Importation du fichier base à l'aide de la commande `mongoimport` hors du Mongo Shell, i.e. directement depuis le terminal. La requête suivante créée une base de données nommée `crimes` avec une collection `crimes_in_london` peuplée par le fichier à l'adresse `./data/base-crimes.json` en mode `merge`, i.e. que si des enregistrements existent, alors ils ne sont pas ajoutés de nouveau; et nous spécifions que le document est un tableau JSON.


`mongoimport --db crimes --collection crimes_in_london --file data/base-crimes.json --mode merge --jsonArray`


<figure>
    <img src="images/mongoimport.png"
         alt="res-mongoimport">
    <figcaption><i>Message de retour de l'importation dans MongoDB.</i></figcaption>
</figure>


Si la base était déjà créée, il faut la supprimer et la recréer depuis le terminal `mongosh`:

- `use crimes`
- `db.dropDatabase()`


### Connexion à la base via PyMongo

In [278]:
from pymongo import MongoClient

adress, port = "localhost", 27017

# Initialisation du client
client = MongoClient(adress, port)
logger.info(f"Connexion établi à MongoDB à l'adresse {adress}:{port}")

# Importation de la base
db = client['crimes']
logger.info("Chargement de la base 'crimes'")

collection = db["crimes_in_london"]
logger.info("Chargement de la collection 'crimes_in_london'")

# Compter les éléments de la collection
nb_document = collection.count_documents({})
logger.info(f"{nb_document} élément{'s' if nb_document > 0 else ''} dans la collection.")

[INFO] - 2026-01-28 20:20:09,027 - __main__ - Connexion établi à MongoDB à l'adresse localhost:27017
[INFO] - 2026-01-28 20:20:09,028 - __main__ - Chargement de la base 'crimes'
[INFO] - 2026-01-28 20:20:09,028 - __main__ - Chargement de la collection 'crimes_in_london'
[INFO] - 2026-01-28 20:20:09,033 - __main__ - 4408 éléments dans la collection.


Il faut que cette dernière ligne renvoie  "5041 éléments dans la collection".

Si une mauvaise manipulation est effectuée pendant l'exécution, il faut supprimer la collection et la peupler de nouveau comme dans la cellule suivante.

In [279]:
with open("data/base-crimes.json", "r") as file:
    crime_data = json.load(file)
    collection.drop()
    collection.insert_many(crime_data)

# 2. Ajout de documents (CREATE)

Le but ici est d'insérer plusieurs enregistrements dans la base. Requêtons à nouveau l'API pour récupérer 10 crimes du même mois mais plus au nord de la zone initiale.

In [280]:
url = "https://data.police.uk/api/crimes-street/all-crime?date=2025-11&poly=51.54669,-0.12634:51.54669,-0.06008:51.52669,-0.06008:51.52669,-0.12634"

try:
    # méthode GET (sans header dans ce cas)
    response = requests.get(url)
except Exception as err:
    logger.error(err)

logger.info(f"La requête a renvoyé {len(list(response))} document{'s' if len(list(response)) > 1 else ''}.")

records = response.json()
for document in records[:10]:
    document["location"]["latitude"] = float(document["location"]["latitude"])
    document["location"]["longitude"] = float(document["location"]["longitude"])

logger.info(f"Conversion des champs latitude/longitude en nombre flottant pour les dix premiers enregistrements.")

first_record, *remaining_records = records[:10]

[INFO] - 2026-01-28 20:20:09,422 - __main__ - La requête a renvoyé 7031 documents.
[INFO] - 2026-01-28 20:20:09,436 - __main__ - Conversion des champs latitude/longitude en nombre flottant pour les dix premiers enregistrements.


Supposons que le premier crime récupéré parmi les nouveaux a effectivement été résolu mais que l'API n'a pas mis-à-jour son statut. Nous réalisons le traitement "à la main" ici et ajoutons cet enregistrement à la base.

In [281]:
# Modification de l'enregistrement
first_record['outcome_status'] = {
    "category": "under-investigation",
    "date": "2025-11"
}

# Insertion du document modifié
res_insert = collection.insert_one(first_record)
logger.info(f"Après insertion du premier enregistrement {res_insert}.")

# Insertion des autres documents
res_insert2 = collection.insert_many(remaining_records)
logger.info(f"Après insertion du premier enregistrement {res_insert2}.")

logger.info(f"{collection.count_documents({})} éléments désormais.")

[INFO] - 2026-01-28 20:20:09,444 - __main__ - Après insertion du premier enregistrement InsertOneResult(ObjectId('697a6169c990f62848e32517'), acknowledged=True).
[INFO] - 2026-01-28 20:20:09,445 - __main__ - Après insertion du premier enregistrement InsertManyResult([ObjectId('697a6169c990f62848e32518'), ObjectId('697a6169c990f62848e32519'), ObjectId('697a6169c990f62848e3251a'), ObjectId('697a6169c990f62848e3251b'), ObjectId('697a6169c990f62848e3251c'), ObjectId('697a6169c990f62848e3251d'), ObjectId('697a6169c990f62848e3251e'), ObjectId('697a6169c990f62848e3251f'), ObjectId('697a6169c990f62848e32520')], acknowledged=True).
[INFO] - 2026-01-28 20:20:09,450 - __main__ - 5051 éléments désormais.


# 3. Interrogation de la collection (READ)

En plus des commandes Python, nous fournissons les commandes à exécuter dans le `MongoShell`. Il faut néanmoins se placer dans la base 'crimes' avec la commande `use crimes`.

D'abord, affichons toutes les catégories de crime une et une seule fois.

`db.crimes_in_london.distinct("category");`

In [282]:
buffer = collection.distinct("category")

logger.info(f"La requête a renvoyé {len(buffer)} valeurs distinctes.")
logger.info(f"Affichage des résultats:\n\t{'\n\t'.join(map(str, buffer))}")

[INFO] - 2026-01-28 20:20:09,464 - __main__ - La requête a renvoyé 14 valeurs distinctes.
[INFO] - 2026-01-28 20:20:09,465 - __main__ - Affichage des résultats:
	anti-social-behaviour
	bicycle-theft
	burglary
	criminal-damage-arson
	drugs
	other-crime
	other-theft
	possession-of-weapons
	public-order
	robbery
	shoplifting
	theft-from-the-person
	vehicle-crime
	violent-crime


Trouvons tous les id des rues concernées par un crime violent: 

`db.crimes_in_london.find({category: "violent-crime"}, {_id: 0, "location.street.id": 1});`

In [283]:
buffer = list(
    collection.find({"category": "violent-crime"}, {"_id": 0, "location.street.id": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,479 - __main__ - La requête a renvoyé 815 documents.
[INFO] - 2026-01-28 20:20:09,480 - __main__ - Affichage des 10 premiers résultats:
	{'location': {'street': {'id': 1682121}}}
	{'location': {'street': {'id': 1678937}}}
	{'location': {'street': {'id': 1688905}}}
	{'location': {'street': {'id': 1688727}}}
	{'location': {'street': {'id': 1681675}}}
	{'location': {'street': {'id': 1490764}}}
	{'location': {'street': {'id': 1687112}}}
	{'location': {'street': {'id': 1688627}}}
	{'location': {'street': {'id': 1686886}}}
	{'location': {'street': {'id': 1685121}}}
...


Affichons tous les lieux de tous les crimes liés au traffic de drogue. La requête affichera l'entité en charge de la sécurité du lieu (BTP ou Force) ainsi que toutes les informations sur le lieu (coordonnées géographiques et rue).

`db.crimes_in_london.find({category: "drugs"}, {_id: 0, location_type: 1, location: 1});`

In [284]:
buffer = list(
    collection.find({"category": "drugs"}, {"_id": 0, "location_type": 1, "location": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,489 - __main__ - La requête a renvoyé 172 documents.
[INFO] - 2026-01-28 20:20:09,490 - __main__ - Affichage des 10 premiers résultats:
	{'location_type': 'Force', 'location': {'latitude': 51.513783, 'street': {'id': 1687855, 'name': "On or near Gower's Walk"}, 'longitude': -0.067839}}
	{'location_type': 'Force', 'location': {'latitude': 51.519775, 'street': {'id': 1687106, 'name': 'On or near Princelet Street'}, 'longitude': -0.071852}}
	{'location_type': 'Force', 'location': {'latitude': 51.497978, 'street': {'id': 1686084, 'name': 'On or near Stevens Street'}, 'longitude': -0.079615}}
	{'location_type': 'Force', 'location': {'latitude': 51.526381, 'street': {'id': 1686172, 'name': 'On or near Nightclub'}, 'longitude': -0.078808}}
	{'location_type': 'Force', 'location': {'latitude': 51.499405, 'street': {'id': 1684841, 'name': 'On or near Staple Street'}, 'longitude': -0.087696}}
	{'location_type': 'Force', 'location': {'latitude': 51.516028, 'street': {'

Trouvons désormais tous les types de crimes ayant une suite d'événements bien formée et l'affichée. 

`db.crimes_in_london.find({outcomes: {$exists: true}}, {_id: 0, category: 1, outcomes: 1});`

In [285]:
buffer = list(
    collection.find({"outcomes": {"$exists": True}}, {"_id": 0, "category": 1, "outcomes": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,507 - __main__ - La requête a renvoyé 4408 documents.
[INFO] - 2026-01-28 20:20:09,508 - __main__ - Affichage des 10 premiers résultats:
	{'category': 'bicycle-theft', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'no-further-action', 'name': 'Investigation complete; no suspect identified'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'bicycle-theft', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'bicycle-theft', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'no-further-action', 'name': 'Investigation complete; no suspect identified'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'bicycle-theft', 'outcomes': [{'category': {'code': 'unde

Trouver tous les crimes avec une suite d'événements bien formés incluant la mention "awaiting-court-result".

    db.crimes_in_london.find(
        {
            outcomes: {
                $exists: true, 
                $elemMatch: {"category.code": "awaiting-court-result"}
            }
        }, 
        {
            _id: 0, 
            category: 1, 
            "location.street.name": 1
        }
    );

In [286]:
buffer = list(
    collection.find({"outcomes": {"$exists": True, "$elemMatch": {"category.code": "awaiting-court-result"}}}, {"_id": 0, "category": 1, "location.street.name": 1})          
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,517 - __main__ - La requête a renvoyé 95 documents.
[INFO] - 2026-01-28 20:20:09,517 - __main__ - Affichage des 10 premiers résultats:
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Watling Street'}}}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Theatre/concert Hall'}}}
	{'category': 'burglary', 'location': {'street': {'name': 'On or near Bethnal Green Road'}}}
	{'category': 'burglary', 'location': {'street': {'name': 'On or near Heneage Place'}}}
	{'category': 'burglary', 'location': {'street': {'name': 'On or near Shopping Area'}}}
	{'category': 'burglary', 'location': {'street': {'name': 'On or near Martin Lane'}}}
	{'category': 'burglary', 'location': {'street': {'name': 'On or near West Poultry Avenue'}}}
	{'category': 'criminal-damage-arson', 'location': {'street': {'name': 'On or near Hospital'}}}
	{'category': 'criminal-damage-arson', 'location': {'street': {'name': 'On or near Police Station

Renvoyons tous les noms de lieu ainsi que la catégorie de crime pour les incidents ayant eu lieu pour un des motifs lié au cambriolage : bicycle-theft, burglary, other-theft, robbery, shoplifting, theft-from-the-person. Additionnellement, ffichons le détail des procédures s'il existe. 

    db.crimes_in_london.find(
        {
            category: {
                $in: ["bicycle-theft", "burglary", "other-theft", "robbery", "shoplifting", "theft-from-the-person"]
            }, 
            outcome_status: {
                $exists: true
            }
        }, 
        {
            _id: 0, 
            category: 1, 
            "location.street.name": 1, 
            "outcome_status.category": 1
        }
    );

In [287]:
buffer = list(
    collection.find({"category": {"$in": ["bicycle-theft", "burglary", "other-theft", "robbery", "shoplifting", "theft-from-the-person"]}, "outcome_status": {"$exists": True}}, {"_id": 0, "category": 1, "location.street.name": 1, "outcome_status.category": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,534 - __main__ - La requête a renvoyé 2915 documents.
[INFO] - 2026-01-28 20:20:09,534 - __main__ - Affichage des 10 premiers résultats:
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Nesham Street'}}, 'outcome_status': {'category': 'Investigation complete; no suspect identified'}}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Theatre/concert Hall'}}, 'outcome_status': {'category': 'Under investigation'}}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Tenter Ground'}}, 'outcome_status': {'category': 'Investigation complete; no suspect identified'}}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Norton Folgate'}}, 'outcome_status': {'category': 'Under investigation'}}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Hospital'}}, 'outcome_status': {'category': 'Investigation complete; no suspect identified'}}
	{'category':

Trouvons la localisation des crimes avec le détail des procédures réalisées s'il s'agit d'un crime classifié comme vol de vélo près du théatre.

`db.crimes_in_london.find({$and: [{outcomes: {$exists: true}, "location.street.name": {$regex: "theatre", $options: "i"}}, {category: {$regex: "^bicycle.*theft", $options: "i"}}]}, {_id: 0, "location.street.name": 1, "location.latitude": 1, "location.longitude": 1});`

In [288]:
buffer = list(
  collection.find({"$and": [{"outcomes": {"$exists": True}, "location.street.name": {"$regex": "theatre", "$options": "i"}}, {"category": {"$regex": "^bicycle.*theft", "$options": "i"}}]}, {"_id": 0, "location.latitude": 1, "location.longitude": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,544 - __main__ - La requête a renvoyé 6 documents.
[INFO] - 2026-01-28 20:20:09,544 - __main__ - Affichage des 10 premiers résultats:
	{'location': {'latitude': 51.519961, 'longitude': -0.09413}}
	{'location': {'latitude': 51.526367, 'longitude': -0.061511}}
	{'location': {'latitude': 51.503345, 'longitude': -0.10179}}
	{'location': {'latitude': 51.520901, 'longitude': -0.094017}}
	{'location': {'latitude': 51.506707, 'longitude': -0.116009}}
	{'location': {'latitude': 51.511794, 'longitude': -0.101906}}
...


Récupérons désormais tous les crimes autour de la cathédrale St. Paul, i.e. tout crime s'étant tenu dans le cadre de longitude $[-0.100844, -0.093409]$ et de latitude $[51.512211, 51.514808]$.

    db.crimes_in_london.find({
        $and:[
            {"location.longitude": {$lte: -0.093409}}, 
            {"location.longitude": {$gte: -0.100844}}, 
            {"location.latitude": {$lte: 51.514808}}, 
            {"location.latitude": {$gte: 51.512211}}
        ]}, 
        {
            _id: 0, 
            category: 1, 
            outcomes: 1
        }
    )

In [289]:
buffer = list(
    collection.find({"$and":[{"location.longitude": {"$lte": -0.093409}}, {"location.longitude": {"$gte": -0.100844}}, {"location.latitude": {"$lte": 51.514808}}, {"location.latitude": {"$gte": 51.512211}}]}, {"_id": 0, "category": 1, "outcomes": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,552 - __main__ - La requête a renvoyé 69 documents.
[INFO] - 2026-01-28 20:20:09,552 - __main__ - Affichage des 10 premiers résultats:
	{'category': 'bicycle-theft', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'charged', 'name': 'Suspect charged'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'awaiting-court-result', 'name': 'Awaiting court outcome'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'criminal-damage-arson', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'local-resolution', 'name': 'Local resolution'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'criminal-damage-arson', 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'categ

Renvoyons toutes les catégories de crimes, avec la localisation et les procédures en cours, telles que le document initial mentionne le mot *court* dans les procédures.


    db.crimes_in_london.find(
        {
            outcomes: {
                $exists: true, 
                $elemMatch: {
                    "category.code": {$regex: /court/} 
                }
            }
        }, 
        {
            _id: 0, 
            category: 1, 
            "location.street.name": 1, 
            outcomes: 1
        }
    );


In [290]:
buffer = list(
    collection.find({"outcomes": {"$exists": True, "$elemMatch": {"category.code": {"$regex": "court"} }}}, {"_id": 0, "category": 1, "location.street.name": 1, "outcomes": 1})
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,562 - __main__ - La requête a renvoyé 95 documents.
[INFO] - 2026-01-28 20:20:09,562 - __main__ - Affichage des 10 premiers résultats:
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Watling Street'}}, 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'charged', 'name': 'Suspect charged'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'awaiting-court-result', 'name': 'Awaiting court outcome'}, 'date': '2025-11', 'person_id': None}]}
	{'category': 'bicycle-theft', 'location': {'street': {'name': 'On or near Theatre/concert Hall'}}, 'outcomes': [{'category': {'code': 'under-investigation', 'name': 'Under investigation'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'charged', 'name': 'Suspect charged'}, 'date': '2025-11', 'person_id': None}, {'category': {'code': 'awaiting-court-result', 'name': 'Awaiti

Récupérons tous les noms des procédures en cours pour les cambriolages.

`db.crimes_in_london.find({category: "burglary", outcomes: {$exists: 1}}, {_id: 0, "outcomes.category.name": 1});`

In [291]:
buffer = list(
    collection.find({"category": "burglary", "outcomes": {"$exists": True} }, {"_id": 0, "outcomes.category.name": 1})              
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des 10 premiers résultats:\n\t{'\n\t'.join(map(str, buffer[:10]))}\n...")

[INFO] - 2026-01-28 20:20:09,570 - __main__ - La requête a renvoyé 224 documents.
[INFO] - 2026-01-28 20:20:09,570 - __main__ - Affichage des 10 premiers résultats:
	{'outcomes': [{'category': {'name': 'Under investigation'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}, {'category': {'name': 'Investigation complete; no suspect identified'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}, {'category': {'name': 'Unable to prosecute suspect'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}, {'category': {'name': 'Investigation complete; no suspect identified'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation'}}, {'category': {'name': 'Investigation complete; no suspect identified'}}]}
	{'outcomes': [{'category': {'name': 'Under investigation

# 4. Mise-à-jour de la collection

Supposons que le document `first_record` qu'on a ajouté précédement doit subir une modification sur son statut de procédure courant, passant à une verbalisation du coupable.

`db.crimes_in_london.updateOne({id: ...}, {$set: {outcome_status: {category: "fined", date: "2025-11"}}})`

(pour cette requête il faut récupérer l'id depuis le notebook car il change à chaque exécution et le document n'a pas de champ `persistent_id`).

In [292]:
id = first_record["id"]

print(id)

132389392


In [293]:
response = collection.update_one({"id": id}, {"$set": {"outcome_status": {"category": "fined", "date": "2025-11"}}})

logger.info(f"La requête a modifé {response.modified_count} document{'s' if response.modified_count>1 else ''} sur {response.matched_count} matché{'s' if response.matched_count>1 else ''}.")

[INFO] - 2026-01-28 20:20:09,580 - __main__ - La requête a modifé 1 document sur 1 matché.


Après réflexion, les dates qui existent dans la base s'étendent qu'en novembre 2025. L'attribut peut donc être supprimé.

`db.crimes_in_london.updateMany({}, {$unset: {month: true}});`

In [294]:
response = collection.update_many({}, {"$unset": {"month": True}})

logger.info(f"La requête a modifé {response.modified_count} document{'s' if response.modified_count>1 else ''} sur {response.matched_count} matché{'s' if response.matched_count>1 else ''}.")

[INFO] - 2026-01-28 20:20:09,630 - __main__ - La requête a modifé 5051 documents sur 5051 matchés.


Une nouvelle directive interne fait passer tous les crimes pour drogues à une autre institution. Il faut donc ajouter une nouvelle ligne dans la liste de procédure pour les cas non-clos relatifs aux drogues. (Motif `action-taken-by-another-organisation`)

    db.crimes_in_london.updateMany(
        {
            $or: [
                {"outcome_status.category": {$regex: "^status.*update.*unavailable.*", $options: "i"}},
                {"outcome_status.category": {$regex: "^under.*investigation.*", $options: "i"}},
                {"outcome_status.category": {$regex: "^awaiting.*court.*result.*", $options: "i"}}
            ]
        }, 
        {
            $push: {
                outcomes: {
                    category: {
                        code: "action-taken-by-another-organisation", 
                        name: "Action to be taken by another organisation"
                    }, 
                    month: "2026-01",
                    person_id: null
                }
            }
        }
    );

In [295]:
response = collection.update_many({"$or": [
    {"outcome_status.category": {"$regex": "^status.*update.*unavailable.*", "$options": "i"}},
    {"outcome_status.category": {"$regex": "^under.*investigation.*", "$options": "i"}},
    {"outcome_status.category": {"$regex": "^awaiting.*court.*result.*", "$options": "i"}}
    ]}, {"$push": {"outcomes": {'category': {'code': 'action-taken-by-another-organisation',
     'name': 'Action to be taken by another organisation'},
    'date': '2026-01',
    'person_id': None}}})

logger.info(f"La requête a modifé {response.modified_count} document{'s' if response.modified_count>1 else ''} sur {response.matched_count} matché{'s' if response.matched_count>1 else ''}.")

[INFO] - 2026-01-28 20:20:09,650 - __main__ - La requête a modifé 1947 documents sur 1947 matchés.


Suite à une directive, il est décidé de supprimer de la base toutes les procédures qui n'ont pas de procédure en cours.

`db.crimes_in_london.deleteMany({"outcome_status.category": null});`

In [296]:
response = collection.delete_many({"outcome_status.category": None})

logger.info(f"La requête a supprimé {response.deleted_count} document{'s' if response.deleted_count>1 else ''}.")

[INFO] - 2026-01-28 20:20:09,658 - __main__ - La requête a supprimé 642 documents.


# 5. Agrégation

Comptons le nombre de documents qui concerne un crime approximativement dans le quartier de la cathédrale St. Paul.

    db.crimes_in_london.find(
        {
            $and: [
                {"location.longitude": {$lte: -0.093409}}, 
                {"location.longitude": {$gte: -0.100844}}, 
                {"location.latitude": {$lte: 51.514808}}, 
                {"location.latitude": {$gte: 51.512211}}
            ]
        }
    ).count();

In [297]:
response = collection.count_documents({"$and":[{"location.longitude": {"$lte": -0.093409}}, {"location.longitude": {"$gte": -0.100844}}, {"location.latitude": {"$lte": 51.514808}}, {"location.latitude": {"$gte": 51.512211}}]})

logger.info(f"La requête a décompté {response} document{'s' if response>1 else ''} pour la requête.")

[INFO] - 2026-01-28 20:20:09,663 - __main__ - La requête a décompté 69 documents pour la requête.


Comptons le nombre de crimes liés à la catégorie "autre crime ou vol"

`db.crimes_in_london.find({category: {$regex: "^other.*", $options: "i"}}).count();`

In [298]:
response = collection.count_documents({"category": {"$regex": "^other.*", "$options": "i"}})

logger.info(f"La requête a décompté {response} document{'s' if response>1 else ''} pour la requête.")

[INFO] - 2026-01-28 20:20:09,670 - __main__ - La requête a décompté 1043 documents pour la requête.


Pour tous les crimes liés à la drogue, écrire le nom de la dernière mise-à-jour de la procédure avec le total associé.

    db.crimes_in_london.aggregate([
      {$match: {category: "drugs"}},
      {$project: {_id: 0, outcomes: 1}},
      {$project: {last_outcome: {$arrayElemAt: ["$outcomes", -1]}}},
      {$group: {_id: "$last_outcome.category.code", total:{ $sum: 1}}}
    ]);

In [299]:
buffer = list(
    collection.aggregate([
      {"$match": {"category": "drugs"}},
      {"$project": {"_id": 0, "outcomes": 1}},
      {"$project": {"last_outcome": {"$arrayElemAt": ["$outcomes", -1]}}},
      {"$group": {"_id": "$last_outcome.category.code", "total":{ "$sum": 1}}}]
))

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des résultats:\n\t{'\n\t'.join(map(str, buffer))}")

[INFO] - 2026-01-28 20:20:09,676 - __main__ - La requête a renvoyé 10 documents.
[INFO] - 2026-01-28 20:20:09,676 - __main__ - Affichage des résultats:
	{'_id': 'unable-to-prosecute', 'total': 34}
	{'_id': 'cautioned', 'total': 2}
	{'_id': 'further-investigation-not-in-public-interest', 'total': 1}
	{'_id': 'under-investigation', 'total': 3}
	{'_id': 'awaiting-court-result', 'total': 12}
	{'_id': 'no-further-action', 'total': 2}
	{'_id': 'formal-action-not-in-public-interest', 'total': 1}
	{'_id': 'penalty-notice-issued', 'total': 3}
	{'_id': 'local-resolution', 'total': 22}
	{'_id': 'action-taken-by-another-organisation', 'total': 92}


Calculer le point central et le chercher sur open-street-map.

    db.crimes_in_london.aggregate([
        {
            $match: {
                $and: [
                    {"location.latitude": {$exists: true}},
                    {"location.longitude": {$exists: true}}
                ]
            }
        },
        {
            $group: {
                _id: null,
                avgLatitude: { $avg: "$location.latitude" },
                avgLongitude: { $avg: "$location.longitude" }
            }
        }
    ]);

In [300]:
buffer = list(
    collection.aggregate([
    {
        "$match": {
            "$and": [
                {"location.latitude": {"$exists": True}},
                {"location.longitude": {"$exists": True}}
            ]
        }
    },{
      "$group": {
        "_id": None,
        "avgLatitude": { "$avg": "$location.latitude" },
        "avgLongitude": { "$avg": "$location.longitude" }
      }
    }
  ])
)

logger.info(f"La requête a renvoyé {len(buffer)} documents.")
logger.info(f"Affichage des résultats:\n\t{'\n\t'.join(map(str, buffer))}")

[INFO] - 2026-01-28 20:20:09,683 - __main__ - La requête a renvoyé 1 documents.
[INFO] - 2026-01-28 20:20:09,683 - __main__ - Affichage des résultats:
	{'_id': None, 'avgLatitude': 51.51264653368111, 'avgLongitude': -0.0975073810387843}


<figure>
    <img src="./images/carte_finale.png"
         alt="image-carte-londres-osm">
    <figcaption><i>Carte centrée sur le point de coordonnées moyennes depuis openstreetmap.</i></figcaption>
</figure>

# 6. MAP REDUCE


Écriture du récapitalutif par classes de crimes avec le décompte.

    db.crimes_in_london.mapReduce(
        function() {
          emit(this.category, 1);
        },
        function(key, values) {
          return Array.sum(values);
        },
        {
          out: "recapitulatif_par_categories"
        }
    );

    db.recapitulatif_par_categories.find({}, {_id: 1, value: 1});

> Notice sur l'usage de l'IA : aucune commande MongoDB / PyMongo n'a été générée par IA. 